This Jupyter Notebook can be used for easy exploration of MariaDB database schema and testing queries interactively. In order to use, make sure to:

Setting up a local database:
1. Install MariaDB 10.5.13-MariaDB from https://mariadb.org/download/?t=mariadb&o=true&p=mariadb&r=10.5.13&os=windows&cpu=x86_64&pkg=msi
2. Open MariaDB's command line interface in your terminal (e.g. "C:\ProgramData\Microsoft\Windows\Start Menu\Programs\MariaDB 10.5 (x64)\MySQL Client (MariaDB 10.5 (x64)).lnk")
3. Create and select a new local database:
    `CREATE DATABASE sodis_app_v1`
    `USE DATABASE sodis_app_v1`
4. Ingest data from sql-dump by just copy/pasting the entire content of "sodis_dbv1_dump.sql" into the console application.


Setting up Python:
1. Install Python (e.g. version 3.7.9)
2. Create a virtual environment 
    `"python -m venv .venv" and activate it ".venv/Scripts/activate"`
3. Install dependencies 
    `"pip install -r requirements.txt"`
4. Make sure to update any environment variables within your local .env file, pointing to your MariaDB database server.
5. Run this notebook

In [11]:
import dotenv
import os
import pandas
import sqlalchemy
import pathlib

%load_ext autoreload
%autoreload 2

dotenv.load_dotenv()
exports_dir = pathlib.Path("exports")
exports_dir.mkdir(exist_ok=True)

print(f"""
    Database server at: '{os.getenv('MARIADB_HOST')}'. 
    Database name is: '{os.getenv('MARIADB_DBNAME')}'.
    """)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload

    Database server at: 'localhost'. 
    Database name is: 'sodis_app_v1'.
    


In [9]:
engine = sqlalchemy.create_engine(f"mysql+pymysql://{os.getenv('MARIADB_USER')}:{os.getenv('MARIADB_KEY')}@{os.getenv('MARIADB_HOST')}/{os.getenv('MARIADB_DBNAME')}")
conn = engine.connect()
tables = pandas.read_sql("SHOW TABLES;", engine)
tables

,Tables_in_sodis_app_v1
0,answer
1,completed_survey
2,input_type
3,interviewee
4,option_choice
5,project
6,question
7,question_image
8,question_option
9,stat


In [29]:
DENORMALIZE_QUERY = """
    SELECT 
        *
    FROM answer
    LEFT JOIN completed_survey
        ON completed_survey.id=answer.completed_survey_id
    LEFT JOIN user
        ON completed_survey.user_id=user.id
    LEFT JOIN interviewee 
        ON interviewee.id=completed_survey.interviewee_id
    LEFT JOIN survey_header
        ON survey_header.id=completed_survey.survey_header_id
    LEFT JOIN project
        ON project.id=survey_header.project_id
    LEFT JOIN village
        ON village.id=village_id
    LEFT JOIN survey_section
        ON survey_header.id=survey_section.survey_header_id
    LEFT JOIN question
        ON question.id=answer.question_id
    LEFT JOIN question_image
        ON question_image.id=question.question_images_id
    LEFT JOIN input_type
        ON input_type.id=question.input_type_id
    LEFT JOIN question_option
        ON answer.question_option_id=question_option.id
        ;
"""

denormalized_database = pandas.read_sql(DENORMALIZE_QUERY, engine)
print(denormalized_database.columns)
denormalized_database.head()

Index(['id', 'question_option_id', 'answer_text', 'completed_survey_id',
       'image_url', 'question_id', 'id', 'interviewee_id', 'survey_header_id',
       'creation_date', 'user_id', 'longitude', 'latitude', 'id', 'first_name',
       'last_name', 'firebase_id', 'email', 'type', 'id', 'name', 'village_id',
       'user_id', 'image_url', 'id', 'survey_name', 'project_id', 'id', 'name',
       'id', 'name', 'id', 'survey_header_id', 'section_name', 'id',
       'survey_section_id', 'input_type_id', 'question_name',
       'dependent_question_id', 'dependent_question_option_id',
       'question_images_id', 'id', 'url', 'id', 'input_type_name', 'id',
       'question_id', 'option_choice_id'],
      dtype='object')


,id,question_option_id,answer_text,completed_survey_id,image_url,question_id,id,interviewee_id,survey_header_id,creation_date,...,dependent_question_id,dependent_question_option_id,question_images_id,id,url,id,input_type_name,id,question_id,option_choice_id
0,02ed3eac-ea95-449d-bdb7-871bdb844529,10146.0,None,0011ccd6-15b5-47e5-80bc-5d527347518a,None,3896,0011ccd6-15b5-47e5-80bc-5d527347518a,079397b5-9f2e-4490-9ebf-795ff2c8c679,515.0,2021-10-27 15:47:26.292,...,NaN,NaN,1183.0,1183.0,https://firebasestorage.googleapis.com/v0/b/ak...,1,single choice,10146.0,3896.0,1267.0
1,09d8a16d-a026-4b68-9ff1-d70da21bd4de,10126.0,None,0011ccd6-15b5-47e5-80bc-5d527347518a,None,3891,0011ccd6-15b5-47e5-80bc-5d527347518a,079397b5-9f2e-4490-9ebf-795ff2c8c679,515.0,2021-10-27 15:47:26.292,...,NaN,NaN,1178.0,1178.0,https://firebasestorage.googleapis.com/v0/b/ak...,1,single choice,10126.0,3891.0,1267.0
2,07d4e063-6283-4381-8486-8db13a35543c,10225.0,None,013524b4-9149-4544-8b44-5e5270e9d8c1,None,3926,013524b4-9149-4544-8b44-5e5270e9d8c1,d3abcc2e-555e-4570-aa93-2420dd2262c5,518.0,2021-08-22 06:25:33.687,...,NaN,NaN,1201.0,1201.0,https://firebasestorage.googleapis.com/v0/b/ak...,1,single choice,10225.0,3926.0,1272.0
3,0a11cf06-6469-4ca8-908d-b68064d7c0dd,9900.0,None,01817771-1a67-4d47-be0d-0ceec54f9bc7,None,3805,01817771-1a67-4d47-be0d-0ceec54f9bc7,5619548e-2457-4b46-b69c-b298aa6708f6,508.0,2021-08-24 12:28:03.0,...,3792.0,9868.0,1116.0,1116.0,https://firebasestorage.googleapis.com/v0/b/ak...,1,single choice,9900.0,3805.0,1235.0
4,0ef9d0ca-f3f0-4994-a416-b635e163f2db,9888.0,None,01817771-1a67-4d47-be0d-0ceec54f9bc7,None,3801,01817771-1a67-4d47-be0d-0ceec54f9bc7,5619548e-2457-4b46-b69c-b298aa6708f6,508.0,2021-08-24 12:28:03.0,...,3792.0,9868.0,1112.0,1112.0,https://firebasestorage.googleapis.com/v0/b/ak...,1,single choice,9888.0,3801.0,1240.0


In [30]:
denormalized_database.to_excel("exports/denormalized_sodis_sqldb.xlsx")

## List of appliedInterventions

multiple surveys linked to one intervention better solved in application code: 

(1) query all applied interventions, 

(2) when creating objects to be written to Amplify, link array of survey objects based on results of query in subsequent cell (surveys_for_intervention_id)

In [10]:

applied_interventions = pandas.read_sql(""" 
    SELECT 
        completed_survey.id AS executed_survey_id,
        project.name as intervention_name, 
        project.id as intervention_id,
        interviewee.id AS interviewee_id, 
        interviewee.name AS interviewee_name,
        user.id AS whoDidIt_id,
        user.first_name AS whoDidIt_first_name,
        user.last_name AS whoDidIt_last_name,
        latitude AS location_latitude,
        longitude AS location_longitude
    FROM completed_survey
    LEFT JOIN user
        ON completed_survey.user_id=user.id
    LEFT JOIN interviewee 
        ON interviewee.id=completed_survey.interviewee_id
    LEFT JOIN survey_header
        ON survey_header.id=completed_survey.survey_header_id
    LEFT JOIN project
        ON project.id=survey_header.project_id
    GROUP BY interviewee_id, intervention_name;
    """,
    engine,
)

applied_interventions

,executed_survey_id,intervention_name,intervention_id,interviewee_id,interviewee_name,whoDidIt_id,whoDidIt_first_name,whoDidIt_last_name,location_latitude,location_longitude
0,f41b562c-9e92-4f09-9ad1-9986eac60173,Ba?o,4,0003f17a-1f22-464b-9393-53d74beffbc3,lorenso sehuenca queno,171,sandalia,baltazar,NaN,NaN
1,988da53a-74f3-4d3c-9cac-b0ee26f90702,Cocina,1,0003f17a-1f22-464b-9393-53d74beffbc3,lorenso sehuenca queno,171,sandalia,baltazar,NaN,NaN
2,a2cfc2e2-b761-4056-b61b-b6f2c05858e0,Datos de la familia,6,0003f17a-1f22-464b-9393-53d74beffbc3,lorenso sehuenca queno,171,sandalia,baltazar,NaN,NaN
3,d1592470-352f-4639-89a1-79ed95bf55b5,Filtro,2,0003f17a-1f22-464b-9393-53d74beffbc3,lorenso sehuenca queno,171,sandalia,baltazar,NaN,NaN
4,90f6ab27-cdd2-4002-ada6-876de4bbd880,Ba?o,4,04325e4d-cce5-4c80-82c6-56ec55597fa5,Edelberto Daga,204,Jose,Yapura Colque,NaN,NaN
5,15122c1b-ff36-4058-ae84-715b91e03c68,Cocina,1,04325e4d-cce5-4c80-82c6-56ec55597fa5,Edelberto Daga,204,Jose,Yapura Colque,NaN,NaN
6,262f3c4e-a616-4aa1-8bea-8c8db9572430,Datos de la familia,6,04325e4d-cce5-4c80-82c6-56ec55597fa5,Edelberto Daga,204,Jose,Yapura Colque,NaN,NaN
7,9dc8d3ab-675d-40c3-8aa7-df95fb5df750,Filtro,2,04325e4d-cce5-4c80-82c6-56ec55597fa5,Edelberto Daga,204,Jose,Yapura Colque,NaN,NaN
8,36333034-96ac-4a21-90ad-08716fdd84cb,Nutrici?n,9,04325e4d-cce5-4c80-82c6-56ec55597fa5,Edelberto Daga,204,Jose,Yapura Colque,NaN,NaN
9,3250a96a-342d-4691-9cb3-3787f170c2a0,Tara,10,04325e4d-cce5-4c80-82c6-56ec55597fa5,Edelberto Daga,204,Jose,Yapura Colque,NaN,NaN


In [ ]:

# applied intervention is uniquely defined based on interviewed person (project_id/interviewee_id) and intervention beneficiary (interviewee_id)
intervention_id = 9
interviewee_id = "079397b5-9f2e-4490-9ebf-795ff2c8c679"

surveys_for_intervention_id = interventions = pandas.read_sql(f""" 
    SELECT * 
    FROM completed_survey
    LEFT JOIN survey_header
        ON survey_header.id=completed_survey.survey_header_id
    WHERE project_id="{intervention_id}"
    AND interviewee_id="{interviewee_id}"
    ;
    """,
    engine,
)

# WHERE completed_survey.id = {SURVEY_ID}
surveys_for_intervention_id

### List of all questions and corresponding answers

In [ ]:
select = """
        question.id AS question_id,
        question_name,
        option_choice_name,
        answer_text,
        dependent_question_option_id,
        dependent_question_id,
        question_images_id,
        input_type_name
        """
# select = "*"

question_answers = pandas.read_sql(f""" 
    SELECT {select}
    FROM answer
    LEFT JOIN question
        ON answer.question_id=question.id
    LEFT JOIN question_option
        ON answer.question_option_id=question_option.id
    LEFT JOIN option_choice
        ON question_option.option_choice_id=option_choice.id
    LEFT JOIN input_type
        ON question.input_type_id=input_type.id
    ;
    """,
    engine,
)

question_answers

## Get questions for a given survey

chain of dependent questions linked to one question better solved in application code: 

(1) run query questions_for_survey_id, 

(2) when creating objects to be written to Amplify, link array of question_options based on result of subsequent query questions_for_question_id

(3) recursively get chain of follow-up question by running query follow_up_question_by_question_id

In [ ]:

unique_survey_ids = pandas.read_sql(f""" 
    SELECT DISTINCT
        id
    FROM survey_header;
    """,
    engine,
).values.tolist()
unique_survey_ids = [unique_survey_ids[i][0] for i in range(len(unique_survey_ids))]
print(f"Unique survey ids are: {unique_survey_ids}")

SURVEY_ID = unique_survey_ids[0]
print(f"Querying all survey data for survey id: {SURVEY_ID}.")


select = """
        question.id AS question_id,
        question_name,
        dependent_question_id,
        dependent_question_option_id,
        dependent_question_option_id,
        dependent_question_id,
        question_images_id,
        input_type_name,
        url AS question_image_url
        """
# select = "*"

questions_for_survey_id = pandas.read_sql(f""" 
    SELECT 
        {select}
    FROM question
    LEFT JOIN survey_section
        ON survey_section.id=question.survey_section_id
    LEFT JOIN survey_header
        ON survey_header.id=survey_section.survey_header_id
    LEFT JOIN question_image
        ON question_image.id=question.question_images_id
    LEFT JOIN input_type
        ON input_type.id=question.input_type_id
    WHERE survey_header.id = {SURVEY_ID};
    """,
    engine,
)

questions_for_survey_id

In [ ]:
select = """
    option_choice_name
"""

select = "*"

QUESTION_ID = 3763

questions_for_question_id = pandas.read_sql(f""" 
    SELECT 
        {select}
    FROM question_option
    LEFT JOIN option_choice
        ON question_option.option_choice_id=option_choice.id
    WHERE question_id={QUESTION_ID}
    ;
    """,
    engine,
)

questions_for_question_id


In [ ]:
DEPENDENT_QUESTION_ID = 3792

follow_up_question_by_question_id = pandas.read_sql(f""" 
    SELECT 
        {select}
    FROM question
    LEFT JOIN survey_section
        ON survey_section.id=question.survey_section_id
    LEFT JOIN survey_header
        ON survey_header.id=survey_section.survey_header_id
    LEFT JOIN question_image
        ON question_image.id=question.question_images_id
    LEFT JOIN input_type
        ON input_type.id=question.input_type_id
    WHERE question.id = {DEPENDENT_QUESTION_ID};
    """,
    engine,
)

follow_up_question_by_question_id